In [2]:
!poetry add -D cupy

Using version ^11.1.0 for cupy

Updating dependencies
Resolving dependencies... (35.5s)Resolving dependencies... (3.6s)Resolving dependencies... (8.2s)Resolving dependencies... (12.6s)Resolving dependencies... (14.4s)Resolving dependencies... (21.3s)Resolving dependencies... (25.8s)

Writing lock file

Package operations: 2 installs, 0 updates, 0 removals

  • Installing fastrlock (0.8): Pending...
  • Installing fastrlock (0.8): Downloading... 0%
  • Installing fastrlock (0.8): Downloading... 100%
  • Installing fastrlock (0.8): Downloading... 100%
  • Installing fastrlock (0.8): Installing...
  • Installing fastrlock (0.8)
  • Installing cupy (11.1.0): Pending...
  • Installing cupy (11.1.0): Downloading... 0%
  • Installing cupy (11.1.0): Downloading... 50%
  • Installing cupy (11.1.0): Downloading... 70%
  • Installing cupy (11.1.0): Downloading... 100%
  • Installing cupy (11.1.0): Downloading... 100%
  • Installing cupy (11.1.0): Installing...
  • Installing cupy (11.1.0)


In [3]:
# An example CUDA Array Interface implementation that wraps a pointer provided
# by cudaMalloc.

from numba import cuda
from ctypes import CDLL, POINTER, byref, c_void_p, c_size_t
import cupy as cp

In [4]:

class MyArray:
    def __init__(self, shape, typestr, data):
        if isinstance(shape, int):
            shape = (shape,)

        self._shape = shape
        self._data = data
        self._typestr = typestr

    @property
    def __cuda_array_interface__(self):
        return {
            'shape': self._shape,
            'typestr': self._typestr,
            'data': (self._data, False),
            'version': 2
        }


In [5]:


# Use ctypes to get the cudaMalloc function from Python
cudart = CDLL('libcudart.so')
cudaMalloc = cudart.cudaMalloc
cudaMalloc.argtypes = [POINTER(c_void_p), c_size_t]

# Allocate some Numba-external memory with cudaMalloc
ptr = c_void_p()
float32_size = 4
nelems = 32
alloc_size = float32_size * nelems
cudaMalloc(byref(ptr), alloc_size)

# Wrap our memory in a CUDA Array Interface object
arr = MyArray(nelems, 'f4', ptr.value)


In [7]:
# Call a kernel on our object wrapping the pointer

@cuda.jit
def initialize(x):
    i = cuda.grid(1)
    if i < len(x):
        x[i] = 3.14 + i


initialize[1, nelems](arr)


# Use CuPy for a convenient way to print our data to show that the kernel
# initialized it
print(cp.asarray(arr))


[ 3.14  4.14  5.14  6.14  7.14  8.14  9.14 10.14 11.14 12.14 13.14 14.14
 15.14 16.14 17.14 18.14 19.14 20.14 21.14 22.14 23.14 24.14 25.14 26.14
 27.14 28.14 29.14 30.14 31.14 32.14 33.14 34.14]


/home/wanghaipeng/.cache/pypoetry/virtualenvs/cuda-player-a56oo6jf-py3.8/lib/python3.8/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
